In [1]:
import numpy as np
import csv
import matplotlib.pyplot as plt
import matplotlib.colors
from matplotlib.colors import ListedColormap
from matplotlib.widgets import CheckButtons
import pandas as pd
import multiprocessing 
import pyodbc
import math
import time
import umap
import datetime
import scipy.spatial.distance as dist
#import hdbscan
from scipy.spatial.distance import cdist
import random
import seaborn as sns
import sklearn.metrics
from IPython.display import Markdown as md
from calendar import timegm
from numba import jit,cuda,vectorize,guvectorize
from tqdm.notebook import tqdm, trange
from sklearn.cluster import KMeans, DBSCAN
from sklearn.neighbors import NearestNeighbors
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,RobustScaler,MinMaxScaler, PolynomialFeatures,LabelEncoder
from sklearn.metrics import silhouette_score,davies_bouldin_score,homogeneity_completeness_v_measure,rand_score,adjusted_rand_score, mean_squared_error, r2_score,normalized_mutual_info_score,fowlkes_mallows_score,plot_confusion_matrix, confusion_matrix, ConfusionMatrixDisplay,classification_report
from sklearn import preprocessing, tree
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression,Ridge,Lasso,LogisticRegression
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.ensemble import RandomForestClassifier, IsolationForest, RandomForestRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import cross_validate


from collections import Counter

#%matplotlib notebook
%matplotlib widget
#%matplotlib inline
plt.rcParams["figure.figsize"] = (3,3)
import torch
from torch import nn


from catboost import CatBoostClassifier
import xgboost as xgb

import geopy.distance
import cmasher as cmr
cmap = cmr.get_sub_cmap('Blues', 0.2, 1)

import warnings
warnings.filterwarnings('ignore')

In [2]:
df_train = pd.read_csv('data/train.csv')
print(df_train.info())
df_train

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [3]:
df_test = pd.read_csv('data/test.csv')
print(df_test.info())
df_test

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB
None


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [4]:
df_train['Sex'] = df_train['Sex'].apply(lambda x: 1 if x =='male' else 0)
df_test['Sex'] = df_test['Sex'].apply(lambda x: 1 if x =='male' else 0)
df_test["Age"].fillna(df_test["Age"].mean(),inplace= True)
# df_test["Fare"].fillna(df_test["Fare"].mean(),inplace= True) 

features = ['Pclass','Age', 'Sex','SibSp','Parch']

df_train = df_train.dropna(subset=features)

In [5]:
X = df_train[features]
y = df_train.Survived

In [6]:
names = [
    "Nearest Neighbors",
    "Decision Tree",
    "Random Forest",
    "Gradient Boosting",
    "Logistic Regression",
    "Neural Network",
    "AdaBoost",
    "CatBoost",
    "XGBoost"
]

max_depth = 4
max_depth2 = max_depth*2

classifiers = [
    KNeighborsClassifier(7),
    DecisionTreeClassifier(max_depth=max_depth),
    RandomForestClassifier(max_depth=max_depth2,max_features=1.0),
    GradientBoostingClassifier(max_depth=20,max_features=1.0),
    LogisticRegression(C=0.9),
    MLPClassifier(alpha=1, max_iter=10000),
    AdaBoostClassifier(),
    CatBoostClassifier(iterations=10,depth=10,loss_function='MultiClass',verbose=False),
    xgb.XGBClassifier(objective='multi:softmax',num_class=2,max_depth=20)
]

N=5

final_train_scores = np.zeros(len(classifiers))
final_val_scores = np.zeros(len(classifiers))
for n in trange(N):
    train_scores = []
    val_scores = []
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
    for name, clf in zip(names, classifiers):
        clf.fit(X_train, y_train)
        train_scores.append(clf.score(X_train, y_train))
        val_scores.append(round(clf.score(X_val, y_val),2))
    final_train_scores += np.array(train_scores)/N
    final_val_scores += np.array(val_scores)/N
for name,trainScore,testScore in zip(names,final_train_scores,final_val_scores):
    print ("{:<21} {:<6.2f} {:<6.2f}".format(name, trainScore, testScore))

  0%|          | 0/5 [00:00<?, ?it/s]

Nearest Neighbors     0.83   0.76  
Decision Tree         0.84   0.79  
Random Forest         0.91   0.82  
Gradient Boosting     0.93   0.80  
Logistic Regression   0.81   0.80  
Neural Network        0.82   0.83  
AdaBoost              0.83   0.82  
CatBoost              0.85   0.81  
XGBoost               0.93   0.81  


In [7]:
model = RandomForestClassifier(max_depth=max_depth2,max_features=1.0)
#model = xgb.XGBClassifier(objective='multi:softmax',num_class=2,max_depth=20)
model.fit(X,y)
model.feature_importances_

array([0.17148096, 0.32244773, 0.39301939, 0.08479808, 0.02825384])

In [8]:
test = df_test[features]
y_pred = model.predict(test)

submission = pd.DataFrame({
    "PassengerId" : df_test["PassengerId"],
    "Survived" : y_pred
})


In [9]:
submission.to_csv('submission.csv', index = False)

In [10]:
submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [11]:
features = ['Pclass', 'Sex','SibSp','Parch']
X = df_train[features]
y = df_train.Age.astype(int)

X = pd.concat([df_train[features], df_test[features]])
y = pd.concat([df_train.Age.astype(int), df_test.Age.astype(int)])

In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor

names = [
    "Decision Tree",
    "Random Forest",
    "Linear Regression",
    "Neural Network"
]

max_depth = 5
max_depth2 = max_depth*2

classifiers = [
    DecisionTreeRegressor(max_depth=max_depth),
    RandomForestRegressor(max_depth=max_depth2,max_features=1.0),
    LinearRegression(),
    MLPRegressor(alpha=1, max_iter=10000)
]

N=5

final_train_scores = np.zeros(len(classifiers))
final_val_scores = np.zeros(len(classifiers))
for n in trange(N):
    train_scores = []
    val_scores = []
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
    for name, clf in zip(names, classifiers):
        clf.fit(X_train, y_train)
        train_scores.append(clf.score(X_train, y_train))
        val_scores.append(round(clf.score(X_val, y_val),2))
    final_train_scores += np.array(train_scores)/N
    final_val_scores += np.array(val_scores)/N
for name,trainScore,testScore in zip(names,final_train_scores,final_val_scores):
    print ("{:<21} {:<6.2f} {:<6.2f}".format(name, trainScore, testScore))

  0%|          | 0/5 [00:00<?, ?it/s]

Decision Tree         0.35   0.25  
Random Forest         0.38   0.23  
Linear Regression     0.21   0.22  
Neural Network        0.29   0.26  


In [13]:
features = ['Pclass', 'Sex','SibSp','Parch']

df_tr = pd.read_csv('data/train.csv')
X_pred_train = df_tr[df_tr.Age.isnull()][features]
X_pred_train['Sex']=X_pred_train['Sex'].apply(lambda x: 1 if x =='male' else 0)

df_te = pd.read_csv('data/test.csv')
X_pred_test = df_te[df_te.Age.isnull()][features]
X_pred_test['Sex']=X_pred_test['Sex'].apply(lambda x: 1 if x =='male' else 0)

In [14]:
model = DecisionTreeRegressor(max_depth=max_depth)
model.fit(X,y)
print(model.feature_importances_)

age_pred_train = model.predict(X_pred_train)
age_pred_test = model.predict(X_pred_test)
len(age_pred_train),len(age_pred_test)

[0.47197051 0.02633909 0.10706657 0.39462383]


(177, 86)

In [15]:
new_ages_train = pd.DataFrame({
    "PassengerId" : df_tr[df_tr.Age.isnull()]["PassengerId"],
    "Age" : age_pred_train
})
new_ages_test = pd.DataFrame({
    "PassengerId" : df_te[df_te.Age.isnull()]["PassengerId"],
    "Age" : age_pred_test
})

In [16]:
df_train2 = pd.read_csv('data/train.csv')
df_test2 = pd.read_csv('data/test.csv')

In [17]:
df_train2.Age = df_train2.Age.fillna(new_ages_train.Age)
df_test2.Age = df_test2.Age.fillna(new_ages_test.Age)
# df_test2.Fare.fillna(df_test2.Fare.mean(),inplace= True)

df_train2['Sex'] = df_train2['Sex'].apply(lambda x: 1 if x =='male' else 0)
df_test2['Sex'] = df_test2['Sex'].apply(lambda x: 1 if x =='male' else 0)

In [18]:
df_train2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    int64  
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(6), object(4)
memory usage: 83.7+ KB


In [19]:
df_test2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    int64  
 4   Age          418 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(5), object(4)
memory usage: 36.0+ KB


In [28]:
features = ['Pclass','Age', 'Sex','SibSp','Parch']
X = df_train2[features]
y = df_train2.Survived

test = df_test2[features]

In [53]:
names = [
    "Nearest Neighbors",
    "Decision Tree",
    "Random Forest",
    "Gradient Boosting",
    "Logistic Regression",
    "Neural Network",
    "AdaBoost",
    #"CatBoost",
    "XGBoost"
]

max_depth = 7
max_depth2 = max_depth*2

classifiers = [
    KNeighborsClassifier(7),
    DecisionTreeClassifier(max_depth=max_depth),
    RandomForestClassifier(max_depth=max_depth2,max_features=0.7),
    GradientBoostingClassifier(max_depth=20,max_features=0.7),
    LogisticRegression(C=0.9),
    MLPClassifier(alpha=1, max_iter=10000),
    AdaBoostClassifier(),
    #CatBoostClassifier(iterations=10,depth=10,loss_function='MultiClass',verbose=False),
    xgb.XGBClassifier(objective='multi:softmax',num_class=2,max_depth=20)
]

N=2

all_pred = []

final_train_scores = np.zeros(len(classifiers))
final_val_scores = np.zeros(len(classifiers))
for n in trange(N):
    train_scores = []
    val_scores = []
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
    for name, clf in zip(names, classifiers):
        clf.fit(X_train, y_train)
        y_pred = clf.predict(test)
        all_pred.append(y_pred)
        train_scores.append(clf.score(X_train, y_train))
        val_scores.append(round(clf.score(X_val, y_val),2))
    final_train_scores += np.array(train_scores)/N
    final_val_scores += np.array(val_scores)/N
for name,trainScore,testScore in zip(names,final_train_scores,final_val_scores):
    print ("{:<21} {:<6.2f} {:<6.2f}".format(name, trainScore, testScore))

  0%|          | 0/2 [00:00<?, ?it/s]

Nearest Neighbors     0.84   0.75  
Decision Tree         0.88   0.80  
Random Forest         0.93   0.80  
Gradient Boosting     0.93   0.78  
Logistic Regression   0.81   0.79  
Neural Network        0.83   0.81  
AdaBoost              0.84   0.81  
XGBoost               0.93   0.80  


In [86]:
ap = np.array(all_pred)/(N*len(names))
apsum = ap.sum(axis=0)
print(np.unique(apsum))

y_pred_all = np.rint(apsum).astype(np.int32)
print(np.unique(y_pred_all))

[0.     0.0625 0.125  0.1875 0.25   0.3125 0.375  0.4375 0.5    0.5625
 0.625  0.6875 0.75   0.8125 0.875  0.9375 1.    ]
[0 1]


In [22]:
model = RandomForestClassifier(max_depth=max_depth2,max_features=1.0)
model.fit(X,y)
print(model.feature_importances_)

model = MLPClassifier(alpha=1, max_iter=10000)
model.fit(X,y)

[0.16237952 0.30460155 0.41968181 0.08200623 0.0313309 ]


MLPClassifier(alpha=1, max_iter=10000)

In [87]:
y_pred = model.predict(test)

submission2 = pd.DataFrame({
    "PassengerId" : df_test["PassengerId"],
    "Survived" : y_pred
})

submission3 = pd.DataFrame({
    "PassengerId" : df_test["PassengerId"],
    "Survived" : y_pred_all
})

In [88]:
submission3.to_csv('submission_final.csv', index = False)

In [82]:
submission.Survived.sum(),submission2.Survived.sum()

(124, 152)

1. Trained a Random Forest Regressor to predict ages on both training and test data.
2. Replaced NaN age values with predictions from the aforementioned RFR. 
3. Trained an MLP Classifier on ['Pclass', 'Age', 'Sex', 'SibSp', 'Parch'].